<a href="https://colab.research.google.com/github/gharaviSara/DataScienceIBM/blob/master/QC_DCM_logit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QUEBEC DCM MODEL

The following examples provide step-by-step instructions to estimate Quebec Discrete choice model for transport mode using Biogeme .


## Install and import `biogeme` package

Install `biogeme` using `pip` as shown below. In addition, import the package and check if GPU processing is available.

In [1]:
!pip install biogeme


In [2]:
!pip install numpy

In [3]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable

## Quebec Dataset

### Read data

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/data/

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks/data


In [5]:
import numpy as np

df_car = pd.read_csv("od_5000_20230404_car.csv")[['id_deplacement','mode','travelDurationTraffic_car']].rename(columns={"travelDurationTraffic_car":"tt_car"})
df_pt = pd.read_csv("od_5000_20230427_pt.csv")[['id_deplacement','mode','travelDuration_pt']].rename(columns={"travelDuration_pt":"tt_pt"})
df_walk = pd.read_csv("od_5000_20230427_walk.csv")[['id_deplacement','mode','travelDuration_walk']].rename(columns={"travelDuration_walk":"tt_walk"})
df_person =  pd.read_csv("od_5000_20230427_person-income.csv" )[['cledeplacement', 'revenu', 'nbveh','age']].rename(columns={"cledeplacement":"id_deplacement"})
df_person = df_person[df_person['id_deplacement'].notna()]

df_person["income_class"] = df_person["revenu"]
df_person.loc[df_person["income_class"].isin([4, 5]), "income_class"] = 4
df_person.loc[df_person["income_class"].isin([6, 7, 8]), "income_class"] = 5
df_person.loc[df_person["income_class"] == 9, "income_class"] = np.random.choice([1, 2, 3, 4, 5],p=[0.20, 0.30, 0.20, 0.15,0.15],size=None, replace=False)



In [6]:
from numpy.core.numeric import ones
df1 = pd.merge(df_car, df_pt , on  = ['id_deplacement','mode'], how='left')
df2 = pd.merge(df1, df_walk , on  = ['id_deplacement','mode'], how='left')
df = pd.merge(df2, df_person , on='id_deplacement', how='left')



In [7]:
df_person.isna().sum()

id_deplacement    0
revenu            0
nbveh             0
age               0
income_class      0
dtype: int64

In [8]:
df_income = df[['id_deplacement','income_class']]
df_income['income_class'] = df_income['income_class'].fillna(1).astype(int)
df_dummies = pd.get_dummies(df_income, columns=['income_class'], prefix='income_class', prefix_sep='_').rename(columns=lambda x: f'inc_{x.split("_")[-1]}')
df = pd.merge(df, df_dummies, left_on='id_deplacement', right_on='inc_deplacement', how='left').drop(columns='inc_deplacement')
df.loc[df["mode"]=='car', "choice"] = 1
df.loc[df["mode"]=='car_passenger', "choice"] = 2
df.loc[df["mode"]=='pt', "choice"] = 3
df.loc[df["mode"]=='walk', "choice"] = 4


<ipython-input-8-5e8f1262ccc4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_income['income_class'] = df_income['income_class'].fillna(1).astype(int)


In [9]:
df.isna().sum()

id_deplacement    0
mode              0
tt_car            0
tt_pt             0
tt_walk           0
revenu            0
nbveh             0
age               0
income_class      0
inc_1             0
inc_2             0
inc_3             0
inc_4             0
inc_5             0
choice            0
dtype: int64

In [10]:
df.loc[df["nbveh"]> 0  , "av1"] = 1
df['av2'] = 1
df['av3'] = 1
df['av4'] = 1
df = df.fillna(0)



In [11]:
df = df.drop(columns='mode')

In [12]:
database = db.Database('odqc', df)


globals().update(database.variables)

In [13]:
df.columns

Index(['id_deplacement', 'tt_car', 'tt_pt', 'tt_walk', 'revenu', 'nbveh',
       'age', 'income_class', 'inc_1', 'inc_2', 'inc_3', 'inc_4', 'inc_5',
       'choice', 'av1', 'av2', 'av3', 'av4'],
      dtype='object')

###Define the data base variable


In [14]:
TT_C = database.DefineVariable('TT_C',  (tt_car /60 ))
TT_PT = database.DefineVariable('TT_PT',  (tt_pt /60 ))
TT_CP = database.DefineVariable('TT_CP',  (tt_car /60 ))
TT_W = database.DefineVariable('TT_W',  (tt_walk /60 ))


In [17]:
VOT1_C   = database.DefineVariable('VOT1_C',  (tt_car /60 ) * inc_1 )
VOT1_CP  = database.DefineVariable('VOT1_CP', (tt_car /60 ) * inc_1 )
VOT1_PT  = database.DefineVariable('VOT1_PT', (tt_pt /60)   * inc_1 )
VOT1_W   = database.DefineVariable('VOT1_W',  (tt_pt /60)   * inc_1 )

VOT2_C   = database.DefineVariable('VOT2_C',  (tt_car /60 ) * inc_2 )
VOT2_CP  = database.DefineVariable('VOT2_CP', (tt_car /60 ) * inc_2 )
VOT2_PT  = database.DefineVariable('VOT2_PT', (tt_pt /60)   * inc_2 )
VOT2_W   = database.DefineVariable('VOT2_W',  (tt_pt /60)   * inc_2 )

VOT3_C   = database.DefineVariable('VOT3_C',  (tt_car /60 ) * inc_4 )
VOT3_CP  = database.DefineVariable('VOT3_CP', (tt_car /60 ) * inc_4 )
VOT3_PT  = database.DefineVariable('VOT3_PT', (tt_pt /60)   * inc_4 )
VOT3_W   = database.DefineVariable('VOT3_W',  (tt_pt /60)   * inc_4 )


VOT4_C   = database.DefineVariable('VOT4_C',  (tt_car /60 ) * inc_4 )
VOT4_CP  = database.DefineVariable('VOT4_CP', (tt_car /60 ) * inc_4 )
VOT4_PT  = database.DefineVariable('VOT4_PT', (tt_pt /60)   * inc_4 )
VOT4_W   = database.DefineVariable('VOT4_W',  (tt_pt /60)   * inc_4 )

VOT5_C   = database.DefineVariable('VOT5_C',  (tt_car /60 ) * inc_5 )
VOT5_CP  = database.DefineVariable('VOT5_CP', (tt_car /60 ) * inc_5 )
VOT5_PT  = database.DefineVariable('VOT5_PT', (tt_pt /60)   * inc_5 )
VOT5_W   = database.DefineVariable('VOT5_W',  (tt_pt /60)   * inc_5 )



In [18]:
AV_C = database.DefineVariable('AV_C', av1 )
AV_CP = database.DefineVariable('AV_CP', av2 )
AV_PT = database.DefineVariable('AV_PT', av3 )
AV_W = database.DefineVariable('AV_W', av4 )

###Parameter to be estimated

ASC (alternative specific constants)

In [19]:
ASC_C  = Beta('ASC_C',  0, None, None, 1)
ASC_CP = Beta('ASC_CP', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_W  = Beta('ASC_W',  0, None, None, 0)

In [20]:
B_TT_C = Beta('B_TT_C', 0, None, None, 0)
B_TT_PT  = Beta('B_TT_PT', 0, None, None, 0)
B_TT_CP = Beta('B_TT_CP', 0, None, None, 0)
B_TT_W  = Beta('B_TT_W', 0, None, None, 0)




In [22]:
B_VOT1_C   = Beta('B_VOT1_C',  0, None, None, 0 )
B_VOT1_CP  = Beta('B_VOT1_CP', 0, None, None, 0 )
B_VOT1_PT  = Beta('B_VOT1_PT', 0, None, None, 0 )
B_VOT1_W   = Beta('B_VOT1_W',  0, None, None, 0 )

B_VOT2_C   = Beta('B_VOT2_C',  0, None, None, 0 )
B_VOT2_CP  = Beta('B_VOT2_CP', 0, None, None, 0 )
B_VOT2_PT  = Beta('B_VOT2_PT', 0, None, None, 0 )
B_VOT2_W   = Beta('B_VOT2_W',  0, None, None, 0 )

B_VOT3_C   = Beta('B_VOT3_C',  0, None, None, 0 )
B_VOT3_CP  = Beta('B_VOT3_CP', 0, None, None, 0 )
B_VOT3_PT  = Beta('B_VOT3_PT', 0, None, None, 0 )
B_VOT3_W   = Beta('B_VOT3_W',  0, None, None, 0 )

B_VOT4_C   = Beta('B_VOT4_C',  0, None, None, 0 )
B_VOT4_CP  = Beta('B_VOT4_CP', 0, None, None, 0 )
B_VOT4_PT  = Beta('B_VOT4_PT', 0, None, None, 0 )
B_VOT4_W   = Beta('B_VOT4_W',  0, None, None, 0 )

B_VOT5_C   = Beta('B_VOT5_C',  0, None, None, 0 )
B_VOT5_CP  = Beta('B_VOT5_CP', 0, None, None, 0 )
B_VOT5_PT  = Beta('B_VOT5_PT', 0, None, None, 0 )
B_VOT5_W   = Beta('B_VOT5_W',  0, None, None, 0 )



###Utility functions


In [23]:
V1 = (ASC_C + B_TT_C * TT_C +B_VOT1_C * VOT1_C + B_VOT2_C * VOT2_C + B_VOT3_C * VOT3_C + B_VOT4_C * VOT4_C + B_VOT5_C * VOT5_C)
V2 = (ASC_CP + B_TT_CP * TT_CP + B_VOT1_CP * VOT1_CP + B_VOT2_CP * VOT2_CP + B_VOT3_CP * VOT3_CP + B_VOT4_CP * VOT4_CP + B_VOT5_CP * VOT5_CP)
V3 = (ASC_PT + B_TT_PT * TT_PT+ B_VOT1_PT * VOT1_PT + B_VOT2_PT * VOT2_PT + B_VOT3_PT * VOT3_PT + B_VOT4_PT* VOT4_PT + B_VOT5_PT * VOT5_PT)
V4 = (ASC_W + B_TT_W * TT_W + B_VOT1_W * VOT1_W + B_VOT2_W * VOT2_W + B_VOT3_W * VOT3_W + B_VOT4_W * VOT4_W + B_VOT5_W * VOT5_W)

In [24]:
# V1 = ( ASC_C +B_TT_C * TT_C)
# V2 = ( ASC_CP +B_TT_CP * TT_CP)
# V3 = (ASC_PT +B_TT_PT * TT_PT)
# V4 = ( ASC_W +B_TT_W * TT_W)


In [25]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4}

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1, 3: 1, 4: 1 }

In [26]:
# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'mnl_quebec'

biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

                  Value   Rob. Std err  Rob. t-test  Rob. p-value
ASC_CP    -1.080784e+00   4.340393e-02   -24.900609  0.000000e+00
ASC_PT    -1.797474e+00   4.569304e-02   -39.338031  0.000000e+00
ASC_W      5.847514e-01   6.001705e-02     9.743087  0.000000e+00
B_INC1     6.376329e-19   1.736916e-15     0.000367  9.997071e-01
B_INC2    -7.918660e-21  1.797693e+308    -0.000000  1.000000e+00
B_INC3    -2.177587e-19   1.649373e-15    -0.000132  9.998947e-01
B_INC4    -2.763893e-19  1.797693e+308    -0.000000  1.000000e+00
B_INC5     1.385435e-19  1.797693e+308     0.000000  1.000000e+00
B_TT_C    -8.142238e-03   5.129509e-03    -1.587333  1.124372e-01
B_TT_CP   -4.048923e-02   6.284992e-03    -6.442210  1.177465e-10
B_TT_PT   -5.916601e-03   1.552215e-03    -3.811714  1.380064e-04
B_TT_W    -6.855762e-02   3.359158e-03   -20.409166  0.000000e+00
B_VOT1_C  -2.221352e-02   7.008273e-03    -3.169615  1.526413e-03
B_VOT1_CP -1.243457e-02   8.267771e-03    -1.503981  1.325863e-01
B_VOT1_PT 